# Transformers 库将目前的 NLP 任务归纳为几下几类：

**文本分类**：例如情感分析、句子对关系判断等；

**对文本中的词语进行分类**：例如词性标注 (POS)、命名实体识别 (NER) 等；

**文本生成**：例如填充预设的模板 (prompt)、预测文本中被遮掩掉 (masked) 的词语；

**从文本中抽取答案**：例如根据给定的问题从一段文本中抽取出对应的答案；

**根据输入文本生成新的句子**：例如文本翻译、自动摘要等。

Transformers 库最基础的对象就是 pipeline() 函数，它封装了预训练模型和对应的前处理和后处理环节。我们只需输入文本，就能得到预期的答案。目前常用的 pipelines 有：

    feature-extraction （获得文本的向量化表示）
    fill-mask （填充被遮盖的词、片段）
    ner（命名实体识别）
    question-answering （自动问答）
    sentiment-analysis （情感分析）
    summarization （自动摘要）
    text-generation （文本生成）
    translation （机器翻译）
    zero-shot-classification （零训练样本分类）

# 情感分析
借助情感分析 pipeline，我们只需要输入文本，就可以得到其情感标签（积极/消极）以及对应的概率：

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("I've been waiting for a HuggingFace course my whole life.")
print(result)
results = classifier(
  ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)
print(results)

pipeline 模型会自动完成以下三个步骤：

    将文本预处理为模型可以理解的格式；
    将预处理好的文本送入模型；
    对模型的预测值进行后处理，输出人类可以理解的格式。

# 零训练样本分类

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
result = classifier(
"This is a course about the Transformers library",
candidate_labels=["education", "politics", "business"],
)
print(result)

# 文本生成
我们首先根据任务需要构建一个模板 提示(prompt)，然后将其送入到模型中来生成后续文本。注意，由于文本生成具有随机性，因此每次运行都会得到不同的结果。


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation")
results = generator("In this course, we will teach you how to")
print(results)
results = generator(
    "In this course, we will teach you how to",
    num_return_sequences=2,
    max_length=50
) 
print(results)

pipeline 自动选择了预训练好的 gpt2 模型来完成任务。我们也可以指定要使用的模型。对于文本生成任务，我们可以在 Model Hub 页面左边选择 Text Generation tag 查询支持的模型。例如，我们在相同的 pipeline 中加载 distilgpt2 模型：

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
results = generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)
print(results)

还可以通过左边的语言 tag 选择其他语言的模型。例如加载专门用于生成中文古诗的 gpt2-chinese-poem 模型：

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="uer/gpt2-chinese-poem")
results = generator(
    "[CLS] 万 叠 春 山 积 雨 晴 ，",
    max_length=40,
    num_return_sequences=2,
)
print(results)

# 遮盖词填充

给定一段部分词语被遮盖掉 (masked) 的文本，使用预训练模型来预测能够填充这些位置的词语。

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
results = unmasker("This course will teach you all about <mask> models.", top_k=2)
print(results)

# 命名实体识别
命名实体识别 (NER) pipeline 负责从文本中抽取出指定类型的实体，例如人物、地点、组织等等。

In [ ]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
results = ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")
print(results)

# 自动问答
自动问答 pipeline 可以根据给定的上下文回答问题

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
answer = question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)
print(answer)

# 自动摘要
自动摘要 pipeline 旨在将长文本压缩成短文本，并且还要尽可能保留原文的主要信息，例如：

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
results = summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
    """
)
print(results)

# 这些 pipeline 背后做了什么？
以第一个情感分析 pipeline 为例
背后经过了三个步骤：

    预处理 (preprocessing)，将原始文本转换为模型可以接受的输入格式；
    将处理好的输入送入模型；
    对模型的输出进行后处理 (postprocessing)，将其转换为人类方便阅读的格式。

## 预处理：使用分词器进行预处理
因为神经网络模型无法直接处理文本，因此首先需要通过预处理环节将文本转换为模型可以理解的数字。具体地，我们会使用每个模型对应的分词器 (tokenizer) 来进行：

    将输入切分为词语、子词或者符号（例如标点符号），统称为 tokens；
    根据模型的词表将每个 token 映射到对应的 token 编号（就是一个数字）；
    根据模型的需要，添加一些额外的输入。

## 将预处理好的输入送入模型
预训练模型的下载方式和分词器 (tokenizer) 类似，Transformers 包提供了一个 AutoModel 类和对应的 from_pretrained() 函数。

预训练模型的本体只包含基础的 Transformer 模块，对于给定的输入，它会输出一些神经元的值，称为 hidden states 或者特征 (features)。对于 NLP 模型来说，可以理解为是文本的高维语义表示。这些 hidden states 通常会被输入到其他的模型部分（称为 head），以完成特定的任务，例如送入到分类头中完成文本分类任务。

## 对模型输出进行后处理
